In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off
!pip install faiss-cpu --progress-bar off
!pip install huggingface-hub --progress-bar off
!pip -q install accelerate --progress-bar off
!pip install llama-cpp-python --progress-bar off
!pip -q install git+https://github.com/huggingface/transformers --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.

In [3]:
import torch
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.vectorstores import FAISS
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [4]:
!rm -rf "db"

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
pdf_file_path = '/content/drive/MyDrive/povs/'
loader = PyPDFDirectoryLoader(pdf_file_path)
docs = loader.load()
len(docs)

22

In [7]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

60

In [9]:
texts

[Document(page_content='An integrated approach to \ncombat cyber risk\nSecuring industrial operations \nin\xa0oil and gas', metadata={'source': '/content/drive/MyDrive/povs/intergrated-approach-combat-cyber-risk-oil-gas.pdf', 'page': 0}),
 Document(page_content='ContentsForeword  1\nIntroduction  2\nUnderstanding the risks  4\nConduct a\xa0maturity assessment  6\nBuild a\xa0unified program  8\nImplement key controls  9\nEmbrace good governance  10\nConclusion  11\nContact us  12', metadata={'source': '/content/drive/MyDrive/povs/intergrated-approach-combat-cyber-risk-oil-gas.pdf', 'page': 1}),
 Document(page_content='Foreword\nThe oil and gas industry is moving into the \nnext stage of evolution, whereby robotics, \ndigitization, and the Internet of Things \n(IoT) are rapidly being integrated into the \noperational environment. The\xa0interest of \ncyber criminals in industrial operations has \nincreased over the last decade resulting in \ncyberattacks that have compromised both \nprod

In [ ]:
import re

# Define a function to remove any non-alphabetic characters from the text
def clean_text(text):
  # Replace any newline characters with spaces
  text = text.replace("\n", " ")
  # Replace any non-breaking spaces with spaces
  text = text.replace("\xa0", " ")
  # Use a regular expression to find and remove any non-alphabetic characters
  text = re.sub("[^a-zA-Z ]", "", text)
  # Return the cleaned text
  return text

# Call the function and print the result
cleaned_texts = clean_text(cleaned_texts)
print(cleaned_texts)


documentpagecontentan integrated approach to combat cyber risk securing industrial operations inoil and gas metadatasource contentdrivemydrivepovsintergratedapproachcombatcyberriskoilgaspdf page  documentpagecontentcontentsforeword  introduction  understanding the risks  conduct amaturity assessment  build aunified program  implement key controls  embrace good governance  conclusion  contact us  metadatasource contentdrivemydrivepovsintergratedapproachcombatcyberriskoilgaspdf page  documentpagecontentforeword the oil and gas industry is moving into the next stage of evolution whereby robotics digitization and the internet of things iot are rapidly being integrated into the operational environment theinterest of cyber criminals in industrial operations has increased over the last decade resulting in cyberattacks that have compromised both production and safety theseattacks have made cyber security ahot discussion topic in boardrooms around the world and now agrowing number of organizati

In [11]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 3.79 s, sys: 306 ms, total: 4.1 s
Wall time: 6.95 s


## Mistral 7b

In [40]:
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/model/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   

In [41]:
DEFAULT_SYSTEM_PROMPT = """
You are a consultant from Deloitte. Answer the questions with details related to the contents found in the text files.""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [42]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat with Multiple PDFs

In [43]:
result = qa_chain("What is deloitte's insight about From bytes to barrels?")
print(result)


llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =      57.93 ms /   107 runs   (    0.54 ms per token,  1847.22 tokens per second)
llama_print_timings: prompt eval time =  305529.71 ms /   574 tokens (  532.28 ms per token,     1.88 tokens per second)
llama_print_timings:        eval time =   72380.18 ms /   106 runs   (  682.83 ms per token,     1.46 tokens per second)
llama_print_timings:       total time =  378654.75 ms /   680 tokens


{'query': "What is deloitte's insight about From bytes to barrels?", 'result': ' Based on the provided context, Deloitte is a professional services firm that provides various services such as audit and assurance, tax and legal, consulting, financial advisory, and risk advisory services. While the context does not mention any specific insights about "From bytes to barrels," Deloitte has a global presence and expertise in a wide range of industries and sectors. Deloitte also emphasizes its commitment to providing industry-leading services that help reinforce public trust in capital markets and enable clients to achieve their goals.', 'source_documents': [Document(page_content='transform and thrive, and lead the way toward a stronger economy, a more equitable society and a sustainable world. Building on its 175 -plus year history, Deloitte \nspans more th an 150 countries and territories. Learn how Deloitte’s more than 345,000 people worldwide make an impact that matters at www.deloitte.c

In [44]:

result = qa_chain("who to be contacted if there is any question about integrated approach to combat cyber risk article?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =     144.46 ms /   256 runs   (    0.56 ms per token,  1772.08 tokens per second)
llama_print_timings: prompt eval time =  298632.93 ms /   582 tokens (  513.12 ms per token,     1.95 tokens per second)
llama_print_timings:        eval time =  177322.37 ms /   255 runs   (  695.38 ms per token,     1.44 tokens per second)
llama_print_timings:       total time =  477466.40 ms /   837 tokens


{'query': 'who to be contacted if there is any question about integrated approach to combat cyber risk article?', 'result': ' If there is any question about the integrated approach to combat cyber risk article, the individuals listed in the text can be contacted:\n\n* Rajeev Chopra: Global Leader – Energy & Resources, Deloitte Touche Tohmatsu Limited, +44 20 7007 2933, rchopra@deloitte.co.uk\n* Steve Livingston: National Power & Utilities – Risk Advisory Leader, Deloitte US, +1 206 716 7539, slivingston@deloitte.com\n* Ramsey Hajj: Risk Advisory – Senior Manager, Deloitte US, +1 561 962 7843, rhajj@deloitte.com\n* Marko Van Zwam: Risk Advisory – Partner, Deloitte Netherlands, +31 88 288 0890, MvanZwam@deloitte.nl\n* Charles Hosner: Risk Advisory – Partner, Deloitte UK, +44 20 7007 ', 'source_documents': [Document(page_content='inherently different, requiring engineering \nknow‑how, and not just IT expertise, in \norder to secure them appropriately.\nToday, an approach that brings toget

In [45]:
len(result["source_documents"])

2

In [46]:
print(result["source_documents"][1].page_content)


+27 11 806 5167
tvanschalkwyk@deloitte.co.zaRajeev Chopra
Global Leader – Energy & Resources
Deloitte Touche Tohmatsu Limited
+44 20 7007 2933
rchopra@deloitte.co.uk
Steve Livingston
National Power & Utilities  
– Risk Advisory Leader
Deloitte US
+1 206 716 7539
slivingston@deloitte.com
Ramsey Hajj
Risk Advisory  
– Senior Manager
Deloitte US
+1 561 962 7843
rhajj@deloitte.com
Marko Van Zwam 
Risk Advisory – Partner
Deloitte Netherlands
+31 88 288 0890
MvanZwam@deloitte.nl
Charles Hosner 
Risk Advisory – Partner
Deloitte UK
+44 20 7007 2827
chosner@deloitte.co.uk
Rob Hayes 
Risk Advisory – Director
Deloitte UK
+44 20 7007 2606
rjhayes@deloitte.co.uk
12An integrated approach to combat cyber risk  | Securing industrial operations in oil and gas


In [47]:
result=qa_chain("What are the challenges of making operational processes secure, vigilant and resilient in the oil and gas industry?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =      29.32 ms /    52 runs   (    0.56 ms per token,  1773.35 tokens per second)
llama_print_timings: prompt eval time =  148067.40 ms /   298 tokens (  496.87 ms per token,     2.01 tokens per second)
llama_print_timings:        eval time =   34414.44 ms /    51 runs   (  674.79 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =  182839.68 ms /   349 tokens


{'query': 'What are the challenges of making operational processes secure, vigilant and resilient in the oil and gas industry?', 'result': ' The challenges of making operational processes secure, vigilant and resilient in the oil and gas industry include harmonizing and aligning two cultures, engineering and IT, and tailoring technical solutions that are not always easy to secure in the operations environment.', 'source_documents': [Document(page_content='An integrated approach to \ncombat cyber risk\nSecuring industrial operations \nin\xa0oil and gas', metadata={'page': 0, 'source': '/content/drive/MyDrive/povs/intergrated-approach-combat-cyber-risk-oil-gas.pdf'}), Document(page_content='Foreword\nThe oil and gas industry is moving into the \nnext stage of evolution, whereby robotics, \ndigitization, and the Internet of Things \n(IoT) are rapidly being integrated into the \noperational environment. The\xa0interest of \ncyber criminals in industrial operations has \nincreased over the 

## Prompts for evaluation

1- What are the main drivers for the shift to integrated reporting across industries and sectors?

2- How does integrated reporting address the disconnect between what companies are disclosing and what stakeholders expect?

3- What are the key components of the Integrated Reporting Framework and how do they help to communicate the value creation of an organization?

4- What are some examples of factors that can affect an organization’s ability to create long-term value for its stakeholders?

6- What are the main reasons why operational systems were not designed with security in mind?

7- How does the IoT increase the vulnerability of industrial facilities to cyber attacks?

8- What are the benefits of adopting an integrated approach to cyber security that brings together IT and engineering?

9- What are some of the goals and practical steps of such an approach?

How does cyber security affect the value creation and preservation of the oil and gas industry?

In [48]:
result=qa_chain("What are the main drivers for the shift to integrated reporting across industries and sectors?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =     145.08 ms /   252 runs   (    0.58 ms per token,  1736.95 tokens per second)
llama_print_timings: prompt eval time =  234457.35 ms /   464 tokens (  505.30 ms per token,     1.98 tokens per second)
llama_print_timings:        eval time =  173496.33 ms /   251 runs   (  691.22 ms per token,     1.45 tokens per second)
llama_print_timings:       total time =  409402.34 ms /   715 tokens


{'query': 'What are the main drivers for the shift to integrated reporting across industries and sectors?', 'result': " Based on the given context, the main drivers for the shift to integrated reporting across industries and sectors are:\n\n1. Stakeholder demands: The evolving expectations and opportunities in today's business ecosystem have created a disconnect between what companies are disclosing and what stakeholders expect in order to make informed decisions. Integrated reporting addresses these stakeholder demands and creates a foundation for future standards in an evolving corporate reporting landscape.\n2. Pressure on resiliency: The paradigm shift in corporate reporting expectations has created pressure on companies to think about the resiliency of their business models and integration of non-financial and financial considerations into their core strategies. This has led many organizations to adopt integrated thinking and integrated reporting.\n3. In response to integrated thi

In [49]:
result=qa_chain("How does integrated reporting address the disconnect between what companies are disclosing and what stakeholders expect?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =     111.60 ms /   209 runs   (    0.53 ms per token,  1872.78 tokens per second)
llama_print_timings: prompt eval time =  124075.32 ms /   246 tokens (  504.37 ms per token,     1.98 tokens per second)
llama_print_timings:        eval time =  142157.37 ms /   208 runs   (  683.45 ms per token,     1.46 tokens per second)
llama_print_timings:       total time =  267372.41 ms /   454 tokens


{'query': 'How does integrated reporting address the disconnect between what companies are disclosing and what stakeholders expect?', 'result': ' Integrated reporting addresses the disconnect between what companies are disclosing and what stakeholders expect by providing a powerful tool that can move the organization away from siloed thinking and reporting and toward integrated thinking, planning, performance, and value management. It explains to providers of financial capital how an organization creates, preserves, or erodes value over time, which is the foundation upon which investors and broader stakeholder groups make informed capital allocation decisions. An integrated report provides coherent and cohesive messaging across all aspects of an organization’s narrative, providing deeper insight into the impact of decisions and business activities, such as what activities drive value creation and which ones do not. It broadens and deepens stakeholder understanding of how an organizatio

In [50]:
result=qa_chain("What are the key components of the Integrated Reporting Framework and how do they help to communicate the value creation of an organization?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =      87.57 ms /   158 runs   (    0.55 ms per token,  1804.35 tokens per second)
llama_print_timings: prompt eval time =  229110.96 ms /   454 tokens (  504.65 ms per token,     1.98 tokens per second)
llama_print_timings:        eval time =  106632.02 ms /   157 runs   (  679.18 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =  336684.13 ms /   611 tokens


{'query': 'What are the key components of the Integrated Reporting Framework and how do they help to communicate the value creation of an organization?', 'result': " The key components of the Integrated Reporting Framework include an approach to creating long-term value, a value creation model, principles for internal and external reporting structure and content, and a process founded on integrated thinking. These components help to communicate the value creation of an organization by providing guidance on how an organization creates, preserves, or erodes value over time and how it can move away from siloed thinking and reporting towards integrated thinking, planning, performance, and value management. The framework also serves as a foundation for corporate reporting that is universal and enduring, providing coherent and cohesive messaging across all aspects of an organization's narrative and broadening and deepening stakeholder understanding of how the organization creates value for i

In [51]:
result=qa_chain("What are some examples of factors that can affect an organization’s ability to create long-term value for its stakeholders?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =      95.07 ms /   157 runs   (    0.61 ms per token,  1651.45 tokens per second)
llama_print_timings: prompt eval time =   89212.62 ms /   174 tokens (  512.72 ms per token,     1.95 tokens per second)
llama_print_timings:        eval time =  106329.85 ms /   156 runs   (  681.60 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =  196406.42 ms /   330 tokens


{'query': 'What are some examples of factors that can affect an organization’s ability to create long-term value for its stakeholders?', 'result': " The factors mentioned in the text are:\n\n1. An effective approach and value creation model established within the organization\n2. The ability of the organization to communicate its strategy, purpose, business model, risks and opportunities effectively\n3. The organization's ability to perform against its strategic objectives in the context of how it creates value for investors and broader stakeholder groups\n4. Internal and external reporting efficiencies\n5. Integrated thinking, which is based on integrated decision making and actions that combined, effect the value creation of an organization\n6. The principles of governance, planet, people, and prosperity (the 4Ps) identified by the World Economic Forum International Business Council’s (WEF-IBC) project on stakeholder capitalism.", 'source_documents': [Document(page_content='approach 

In [52]:
result=qa_chain("What are the main reasons why operational systems were not designed with security in mind?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =      89.76 ms /   159 runs   (    0.56 ms per token,  1771.41 tokens per second)
llama_print_timings: prompt eval time =  277971.93 ms /   549 tokens (  506.32 ms per token,     1.98 tokens per second)
llama_print_timings:        eval time =  108231.52 ms /   158 runs   (  685.01 ms per token,     1.46 tokens per second)
llama_print_timings:       total time =  387204.01 ms /   707 tokens


{'query': 'What are the main reasons why operational systems were not designed with security in mind?', 'result': ' The main reasons why operational systems were not designed with security in mind are:\n\n1. Little need for security: Originally, there was little need for security as fit-for-purpose, isolated operational systems were the order of the day, and the risk of a large-scale cascading failure due to an attack was extremely isolated.\n2. Lack of connectivity: The digitization of operational processes has led to new opportunities to improve productivity and drive down costs, but the convergence of operational and business systems has also opened the enterprise to a whole new array of cyber risks. The ubiquitous connectivity of the Internet of Things (IoT) has turned the most basic assumptions about operational security upside down, making it difficult to secure ICS.', 'source_documents': [Document(page_content='Critical infrastructure relies on industrial \ncontrol systems (ICS)

In [53]:
result=qa_chain("How does the IoT increase the vulnerability of industrial facilities to cyber attacks?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =      72.88 ms /   126 runs   (    0.58 ms per token,  1728.82 tokens per second)
llama_print_timings: prompt eval time =  138957.42 ms /   271 tokens (  512.76 ms per token,     1.95 tokens per second)
llama_print_timings:        eval time =   86356.95 ms /   125 runs   (  690.86 ms per token,     1.45 tokens per second)
llama_print_timings:       total time =  226053.98 ms /   396 tokens


{'query': 'How does the IoT increase the vulnerability of industrial facilities to cyber attacks?', 'result': ' The IoT increases the vulnerability of industrial facilities to cyber attacks by connecting these facilities to a global network that is accessible from anywhere in the world. This allows cybercriminals to gain remote access to programmable logic controllers (PLCs) and disrupt production processes at will. The ubiquitous connectivity also increases the risk of cyber attacks from various sources, including internal actors aiming to sabotage production, competitors seeking to cause brand damage, and external parties wanting to shut down operations. Additionally, behavioral aspects such as a lack of security awareness within the organization can also contribute to vulnerabilities.', 'source_documents': [Document(page_content='Critical infrastructure relies on industrial \ncontrol systems (ICS) to maintain safe \nand reliable operations. Engineers\xa0have \nsuccessfully designed 

In [54]:
result=qa_chain("What are the benefits of adopting an integrated approach to cyber security that brings together IT and engineering?")
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4160.70 ms
llama_print_timings:      sample time =     119.62 ms /   213 runs   (    0.56 ms per token,  1780.65 tokens per second)
llama_print_timings: prompt eval time =  239765.41 ms /   477 tokens (  502.65 ms per token,     1.99 tokens per second)
llama_print_timings:        eval time =  145404.04 ms /   212 runs   (  685.87 ms per token,     1.46 tokens per second)
llama_print_timings:       total time =  386420.71 ms /   689 tokens


{'query': 'What are the benefits of adopting an integrated approach to cyber security that brings together IT and engineering?', 'result': ' The benefits of adopting an integrated approach to cyber security that brings together IT and engineering in the oil and gas industry include:\n\n1. Addressing cyber risks: The oil and gas industry faces various cyber risks such as production and safety compromises due to cyberattacks. An integrated approach can help identify and mitigate these risks effectively.\n2. Harmonizing engineering and IT cultures: The oil and gas industry involves both engineering and IT processes, which need to be harmonized and aligned for effective security measures. An integrated approach can facilitate this process by bringing together these two cultures.\n3. Tailored technical solutions: The operations environment in the oil and gas industry demands tailored technical solutions that are not always easy to secure. An integrated approach can provide customized techni